In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import datetime
from datetime import datetime
from tqdm.notebook import tqdm
import time
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import date_format
from pyspark.sql.functions import *
import requests
from telethon.sync import TelegramClient

date1 = input('Enter a date in YYYY-MM-DD format ')                                     ## TO BE CHANGED DAILY AS PER UPDATION DATE
year , month , day = map(int,date1.split('-'))
day , month = f"{day:02d}",f"{month:02d}"
tablename="r"+str(day)+str(month)+str(year)
print(tablename)

st = time.time()
def banknifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.1.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    banknifty = q.filter(q.ticker.contains('BANKNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    banknifty = banknifty.withColumn('ticker',regexp_replace('ticker','.NFO',''))

    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    banknifty = banknifty.toPandas()
    banknifty = banknifty.rename(columns={'ticker':'Ticker','date':'Date','time':'Time','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})
    
    # df = pd.read_csv(r"C:\\users\\admin\\desktop\\NSEFO_06092023.csv")
    # banknifty = df[(df['Ticker'].str.contains('BANKNIFTY') & (df['Ticker'].str.endswith('E.NFO')))].reset_index(drop=True)
    # banknifty['Ticker'] = banknifty['Ticker'].str.replace('.NFO','')
    
    banknifty = banknifty[(banknifty['Time']>='09:15:59') & (banknifty['Time']<='15:29:59')].reset_index(drop=True)
    banknifty['Date'] = pd.to_datetime(banknifty['Date'],format='mixed',dayfirst=True)
    banknifty['Temp'] = banknifty['Ticker'].str.replace('BANKNIFTY','').str.replace('.NFO','').str[:-2]
    banknifty['Length_Of_Temp'] = np.where(banknifty['Temp'].str.len()==12,12,banknifty['Temp'].str.len())
    banknifty['Ticker_Expiry_Date'] = pd.to_datetime(banknifty['Temp'].str[:7],format='mixed',dayfirst=True)
    banknifty['Exp_year'] = banknifty['Ticker_Expiry_Date'].dt.year
    banknifty['Exp_month'] = banknifty['Ticker_Expiry_Date'].dt.month
    banknifty = banknifty[banknifty['Ticker_Expiry_Date']>=banknifty['Date']].reset_index(drop=True)                             ## filtering out wrong ticker dates
    banknifty['DayOfWeek'] = banknifty['Date'].dt.isocalendar().day
    banknifty['ExpiryDayOfWeek'] = banknifty['Ticker_Expiry_Date'].dt.isocalendar().day
    banknifty['Current_Week_number'] = banknifty['Date'].dt.isocalendar().week
    banknifty['Expiry_Week_number_check'] = banknifty['Ticker_Expiry_Date'].dt.isocalendar().week
    # banknifty.to_csv(r'C:\\users\\admin\\desktop\\banknifty1.csv',index=False)
    banknifty['Expiry_Week_number'] = np.where(banknifty['ExpiryDayOfWeek']<=3,banknifty['Ticker_Expiry_Date'].dt.isocalendar().week-1,banknifty['Ticker_Expiry_Date'].dt.isocalendar().week)
    banknifty['Expiry_Week_number'] = np.where((banknifty['DayOfWeek']<=3) & (banknifty['ExpiryDayOfWeek']<=3),banknifty['Expiry_Week_number']+1,banknifty['Expiry_Week_number'])
    banknifty['Week_Diff'] = banknifty['Expiry_Week_number'] - banknifty['Current_Week_number']
    banknifty["Week_Diff"] = banknifty['Week_Diff'].replace(np.nan,10000)
    return banknifty

def banknifty_weekly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Weekly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Weekly\\'+files)
    
    banknifty = banknifty_data()
    weekly1 = banknifty[banknifty['ExpiryDayOfWeek']<=3]
    weekly2 = banknifty[banknifty['ExpiryDayOfWeek']==4]
    agb = weekly1.groupby(["Week_Diff"])
    unique_val_list_a = list(weekly1["Week_Diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    for i in sorted(unique_val_list_a):
        if i <= 14:
            print(f'Week{i+1}')
            temp_df = agb.get_group(i)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V',np.where(i==5,'VI',np.where(i==6,'VII',np.where(i==7,'VIII',np.where(i==8,'IX',np.where(i==9,'X',np.where(i==10,'XI',np.where(i==11,'XII',np.where(i==12,'XIII',np.where(i==13,'XIV',''))))))))))))))
            temp_df['Symbol'] = 'BANKNIFTY' + 'WEEKLY-' + str(file) + temp_df['Temp'].str[-5:].astype(int).astype(str) + temp_df['Ticker'].str[-2:]
            temp_df['Ticker'] = temp_df['Symbol']
            temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
            temp_df = temp_df.drop_duplicates()
            temp_df.to_csv(fr'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Weekly\\BANKNIFTY-{file}.csv',index=False)
    print("\n")
    agb = weekly2.groupby(["Week_Diff"])
    unique_val_list_a = list(weekly2["Week_Diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    for i in sorted(unique_val_list_a):
        if i <= 14:
            print(f"Week{i+1}")
            temp_df = agb.get_group(i)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V',np.where(i==5,'VI',np.where(i==6,'VII',np.where(i==7,'VIII',np.where(i==8,'IX',np.where(i==9,'X',np.where(i==10,'XI',np.where(i==11,'XII',np.where(i==12,'XIII',np.where(i==13,'XIV',''))))))))))))))
            temp_df['Symbol'] = 'BANKNIFTY' + 'WEEKLY-' + str(file) + temp_df['Temp'].str[-5:].astype(int).astype(str) + temp_df['Ticker'].str[-2:]
            temp_df['Ticker'] = temp_df['Symbol']
            temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
            temp_df = temp_df.drop_duplicates()
            temp_df.to_csv(fr'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Weekly\\BANKNIFTY-{file}.csv',index=False)
    print("Weekly Contracts Created!")

def banknifty_monthly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Monthly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Monthly\\'+files)
    
    banknifty = banknifty_data()
    i = 0
    monthly_df = pd.DataFrame()
    for name,group in tqdm(banknifty.groupby(['Exp_year','Exp_month'])):
        monthly_df = pd.concat([monthly_df,group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]],ignore_index=True)
        if i <3 :
            print(f"Monthly{i+1}")
            monthly = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(monthly)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III','')))
            monthly['Symbol'] = 'BANKNIFTY' + 'MONTHLY-' + str(file) + monthly['Temp'].str[-5:].astype(int).astype(str) + monthly['Ticker'].str[-2:]
            monthly['Ticker'] = monthly['Symbol']
            monthly = monthly.drop(monthly.columns[9:],axis=1)
            monthly.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Monthly\\Banknifty-{file}.csv',index=False)
            i+=1
    print("Monthly Contracts Created")
    return monthly_df

def banknifty_quarterly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Quarterly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Quarterly\\'+files)
        
    quarterly = banknifty_monthly()
    quarterly = quarterly[(quarterly['Exp_month'] == 3) | (quarterly['Exp_month'] == 6) | (quarterly['Exp_month'] == 9) | (quarterly['Exp_month'] == 12)].reset_index(drop=True)
    quarter = 0
    quarter_check = 3
    quarterly_final_df = pd.DataFrame()
    for name, group in tqdm(quarterly.groupby(['Exp_year','Exp_month'])):
        quarterly_final_df = pd.concat([quarterly_final_df,group],ignore_index=True)
        # print(name)
        if quarter == 0:
            print(f"Quarter{quarter+1}")
            quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(quarterly_df)
            file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
            quarterly_df['Symbol'] = 'BANKNIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
            quarterly_df['Ticker'] = quarterly_df['Symbol']
            quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
            quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Quarterly\\Banknifty-{file}.csv',index=False)
            quarter_month_counter = group.iloc[0]['Exp_month']
            quarter_year_counter = group.iloc[0]['Exp_year']
            quarter +=1
            
        else:
            if quarter_month_counter == 12:
                quarter_month_counter = 0
            if group.iloc[0]['Exp_month'] - quarter_month_counter == quarter_check:
                print(f"Quarter{quarter+1}")
                quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                # display(quarterly_df)
                file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
                quarterly_df['Symbol'] = 'BANKNIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
                quarterly_df['Ticker'] = quarterly_df['Symbol']
                quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
                quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Quarterly\\Banknifty-{file}.csv',index=False)
                quarter_month_counter = group.iloc[0]['Exp_month']
                quarter_year_counter = group.iloc[0]['Exp_year']
                quarter +=1
            else:
                while(quarter<4):
                    quarter +=1
                    quarter_check+=3
                    if group.iloc[0]['Exp_month'] - quarter_month_counter == quarter_check:
                        print(f"Quarter{quarter+1}")
                        quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                        # display(quarterly_df)
                        file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
                        quarterly_df['Symbol'] = 'BANKNIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
                        quarterly_df['Ticker'] = quarterly_df['Symbol']
                        quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
                        quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Quarterly\\Banknifty-{file}.csv',index=False)
                        quarter_month_counter = group.iloc[0]['Exp_month']
                        quarter_year_counter = group.iloc[0]['Exp_year']
                        quarter +=1
    print("Quarterly Contracts Created")
    return quarterly_final_df

def banknifty_halfyearly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\'+files)
        
    half_yearly = banknifty_quarterly()
    half_yearly = half_yearly[(half_yearly['Exp_month'] == 6) | (half_yearly['Exp_month'] == 12)].reset_index(drop=True)
    half_year = 0
    halfyear_check = 6
    for name, group in tqdm(half_yearly.groupby(['Exp_year','Exp_month'])):
        # print(name)
        
        if half_year == 0:
            print(f"HalfYearly{half_year+1}")
            half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(half_yearly_df)
            half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
            half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_year+1}.csv',index=False)
            halfyear_month_counter = group.iloc[0]['Exp_month']
            halfyear_year_counter = group.iloc[0]['Exp_year']
            half_year +=1
        else:
            if halfyear_month_counter == 12:
                halfyear_month_counter = 0
            if group.iloc[0]['Exp_month'] - halfyear_month_counter == halfyear_check:
                print(f"HalfYearly{half_year+1}")
                half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                # display(half_yearly_df)
                half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
                half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_year+1}.csv',index=False)
                halfyear_month_counter = group.iloc[0]['Exp_month']
                halfyear_year_counter = group.iloc[0]['Exp_year']
                half_year +=1
            else:
                while(half_year<10):
                    half_year +=1
                    halfyear_check+=6
                    if group.iloc[0]['Exp_month'] - halfyear_month_counter == halfyear_check:
                        print(f"HalfYearly{half_year+1}")
                        half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                        # display(half_yearly_df)
                        half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
                        half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_year+1}.csv',index=False)
                        halfyear_month_counter = group.iloc[0]['Exp_month']
                        halfyear_year_counter = group.iloc[0]['Exp_year']
                        halfyear_check=0
                        break
    print("Half Yearly contracts created!")

def banknifty_yearly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Yearly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Yearly\\'+files)
        
    yearly_file = 1
    half_yearly_file1 = 1
    half_yearly_file2 = 2
    for i in range(5):
        # print(i,yearly_file,half_yearly_file1,half_yearly_file2)
        if os.path.exists(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_yearly_file1}.csv'):
            df1 = pd.read_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_yearly_file1}.csv')
        else:
            df1 = pd.DataFrame()
        if os.path.exists(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_yearly_file2}.csv'):
            df2 = pd.read_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{half_yearly_file2}.csv')
        else:
            df2 = pd.DataFrame()
        if df1.empty and df2.empty == True:
            print(f"No Contracts for HalfYearly {half_yearly_file1} {half_yearly_file2}")
        else:
            final_df = pd.DataFrame()
            if not df1.empty:
                df1['Temp'] = df1['Ticker'].str.replace('BANKNIFTY','').str.replace('.NFO','').str[:-2]
                df1['Ticker_Expiry_Date'] = pd.to_datetime(df1['Temp'].str[:7],format='mixed',dayfirst=True)
                df1['Exp_month'] = df1['Ticker_Expiry_Date'].dt.month
                df1 = df1[df1['Exp_month']==12].reset_index(drop=True)
            if not df2.empty:
                df2['Temp'] = df2['Ticker'].str.replace('BANKNIFTY','').str.replace('.NFO','').str[:-2]
                df2['Ticker_Expiry_Date'] = pd.to_datetime(df2['Temp'].str[:7],format='mixed',dayfirst=True)
                df2['Exp_month'] = df2['Ticker_Expiry_Date'].dt.month
                df2 = df2[df2['Exp_month']==12].reset_index(drop=True)
            final_df = pd.concat([df1,df2],ignore_index=True)
            
            if not df1.empty:
                file1 = np.where(half_yearly_file1==1,'I',np.where(half_yearly_file1==3,'III',np.where(half_yearly_file1==5,'V',np.where(half_yearly_file1==7,'VII',np.where(half_yearly_file1==9,'IX','')))))
                df1['Symbol'] = 'BANKNIFTY-' + str(file1) + df1['Temp'].str[-5:].astype(int).astype(str) + df1['Ticker'].str[-2:]
                df1['Ticker'] = df1['Symbol']
                df1 = df1.drop(df1.columns[9:],axis=1)
                df1.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{file1}.csv',index=False)
            
            if not df2.empty:
                file2 = np.where(half_yearly_file2==2,'II',np.where(half_yearly_file2==4,'IV',np.where(half_yearly_file2==6,'VI',np.where(half_yearly_file2==8,'VIII',np.where(half_yearly_file2==10,'X','')))))
                df2['Symbol'] = 'BANKNIFTY-' + str(file2) + df2['Temp'].str[-5:].astype(int).astype(str) + df2['Ticker'].str[-2:]
                df2['Ticker'] = df2['Symbol']
                df2 = df2.drop(df2.columns[9:],axis=1)
                df2.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Half_Yearly\\Banknifty-{file2}.csv',index=False)
                
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V','')))))
            final_df['Symbol'] = 'BANKNIFTY-' + str(file) + final_df['Temp'].str[-5:] + final_df['Ticker'].str[-2:]
            final_df['Ticker'] = final_df['Symbol']
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            final_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\BankNifty\\Yearly\\BankNifty-{file}.csv',index=False)
        yearly_file+=1
        half_yearly_file1+=2
        half_yearly_file2+=2
    print("Yearly Contracts Created")

def nifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.1.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
     = q.filter(q.ticker.contains('NIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    nifty = nifty.withColumn('ticker_check',substring('ticker',1,5))
    nifty = nifty.filter(nifty.ticker_check.contains('NIFTY'))
    ## REPLACING .NFO IN ticker
    nifty = nifty.withColumn('ticker',regexp_replace('ticker','.NFO',''))
    nifty = nifty.drop(col('ticker_check'))

    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    nifty = nifty.toPandas()
    nifty = nifty.rename(columns={'ticker':'Ticker','date':'Date','time':'Time','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})
    
    # df = pd.read_csv(r"C:\\users\\admin\\desktop\\NSEFO_06092023.csv")
    # nifty = df[(df['Ticker'].str.startswith('NIFTY') & (df['Ticker'].str.endswith('E.NFO')))].reset_index(drop=True)
    # nifty['Ticker'] = nifty['Ticker'].str.replace('.NFO','')
    
    nifty = nifty[(nifty['Time']>='09:15:59') & (nifty['Time']<='15:29:59')].reset_index(drop=True)
    nifty['Date'] = pd.to_datetime(nifty['Date'],format='mixed',dayfirst=True)
    nifty['Temp'] = nifty['Ticker'].str.replace('NIFTY','').str.replace('.NFO','').str[:-2]
    nifty['Length_Of_Temp'] = np.where(nifty['Temp'].str.len()==12,12,nifty['Temp'].str.len())
    nifty['Ticker_Expiry_Date'] = pd.to_datetime(nifty['Temp'].str[:7],format='mixed',dayfirst=True)
    nifty['Exp_year'] = nifty['Ticker_Expiry_Date'].dt.year
    nifty['Exp_month'] = nifty['Ticker_Expiry_Date'].dt.month
    nifty = nifty[nifty['Ticker_Expiry_Date']>=nifty['Date']].reset_index(drop=True)                             ## filtering out wrong ticker dates
    nifty['DayOfWeek'] = nifty['Date'].dt.isocalendar().day
    nifty['ExpiryDayOfWeek'] = nifty['Ticker_Expiry_Date'].dt.isocalendar().day
    nifty['Current_Week_number'] = nifty['Date'].dt.isocalendar().week
    nifty['Expiry_Week_number_check'] = nifty['Ticker_Expiry_Date'].dt.isocalendar().week
    # banknifty.to_csv(r'C:\\users\\admin\\desktop\\banknifty1.csv',index=False)
    nifty['Expiry_Week_number'] = np.where(nifty['ExpiryDayOfWeek']<=4,nifty['Ticker_Expiry_Date'].dt.isocalendar().week-1,nifty['Ticker_Expiry_Date'].dt.isocalendar().week)
    nifty['Expiry_Week_number'] = np.where((nifty['DayOfWeek']<=4) & (nifty['ExpiryDayOfWeek']<=4),nifty['Expiry_Week_number']+1,nifty['Expiry_Week_number'])
    nifty['Week_Diff'] = nifty['Expiry_Week_number'] - nifty['Current_Week_number']
    nifty["Week_Diff"] = nifty['Week_Diff'].replace(np.nan,10000)
    return nifty

def nifty_weekly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Weekly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Weekly\\'+files)
        
    nifty = nifty_data()
    weekly1 = nifty[nifty['ExpiryDayOfWeek']<=4].reset_index(drop=True)
    agb = weekly1.groupby(["Week_Diff"])
    unique_val_list_a = list(weekly1["Week_Diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    for i in sorted(unique_val_list_a):
        if i <= 14:
            print(f'Week{i+1}')
            temp_df = agb.get_group(i)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V',np.where(i==5,'VI',np.where(i==6,'VII',np.where(i==7,'VIII',np.where(i==8,'IX',np.where(i==9,'X',np.where(i==10,'XI',np.where(i==11,'XII',np.where(i==12,'XIII',np.where(i==13,'XIV',''))))))))))))))
            temp_df['Symbol'] = 'NIFTY' + 'WEEKLY-' + str(file) + temp_df['Temp'].str[-5:].astype(int).astype(str) + temp_df['Ticker'].str[-2:]
            temp_df['Ticker'] = temp_df['Symbol']
            temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
            temp_df = temp_df.drop_duplicates()
            temp_df.to_csv(fr'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Weekly\\NIFTY-{file}.csv',index=False)
    print("Weekly Contracts Created")

def nifty_monthly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Monthly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Monthly\\'+files)
        
    nifty = nifty_data()
    i = 0
    monthly_df = pd.DataFrame()
    for name,group in tqdm(nifty.groupby(['Exp_year','Exp_month'])):
        monthly_df = pd.concat([monthly_df,group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]],ignore_index=True)
        if i <3 :
            print(f"Monthly{i+1}")
            monthly = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(monthly)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III','')))
            monthly['Symbol'] = 'NIFTY' + 'MONTHLY-' + str(file) + monthly['Temp'].str[-5:].astype(int).astype(str) + monthly['Ticker'].str[-2:]
            monthly['Ticker'] = monthly['Symbol']
            monthly = monthly.drop(monthly.columns[9:],axis=1)
            monthly.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Monthly\\NIFTY-{file}.csv',index=False)
            i+=1
    print("Monthly Contracts Created")
    return monthly_df

def nifty_quarterly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Quarterly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Quarterly\\'+files)
        
    quarterly = nifty_monthly()
    quarterly = quarterly[(quarterly['Exp_month'] == 3) | (quarterly['Exp_month'] == 6) | (quarterly['Exp_month'] == 9) | (quarterly['Exp_month'] == 12)].reset_index(drop=True)
    quarter = 0
    quarter_check = 3
    quarterly_final_df = pd.DataFrame()
    for name, group in tqdm(quarterly.groupby(['Exp_year','Exp_month'])):
        quarterly_final_df = pd.concat([quarterly_final_df,group],ignore_index=True)
        # print(name)
        if quarter == 0:
            print(f"Quarter{quarter+1}")
            file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
            quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(quarterly_df)
            quarterly_df['Symbol'] = 'NIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
            quarterly_df['Ticker'] = quarterly_df['Symbol']
            quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
            quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Quarterly\\NIFTY-{file}.csv',index=False)
            quarter_month_counter = group.iloc[0]['Exp_month']
            quarter_year_counter = group.iloc[0]['Exp_year']
            quarter +=1
            
        else:
            if quarter_month_counter == 12:
                quarter_month_counter = 0
            if group.iloc[0]['Exp_month'] - quarter_month_counter == quarter_check:
                print(f"Quarter{quarter+1}")
                file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
                quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                quarterly_df['Symbol'] = 'NIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
                quarterly_df['Ticker'] = quarterly_df['Symbol']
                # display(quarterly_df)
                quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
                quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Quarterly\\NIFTY-{file}.csv',index=False)
                quarter_month_counter = group.iloc[0]['Exp_month']
                quarter_year_counter = group.iloc[0]['Exp_year']
                quarter +=1
            else:
                while(quarter<4):
                    quarter +=1
                    quarter_check+=3
                    if group.iloc[0]['Exp_month'] - quarter_month_counter == quarter_check:
                        print(f"Quarter{quarter+1}")
                        file = np.where(quarter==0,'I',np.where(quarter==1,'II',np.where(quarter==2,'III',np.where(quarter==3,'IV',''))))
                        quarterly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                        quarterly_df['Symbol'] = 'NIFTY' + 'QUARTERLY-' + str(file) + quarterly_df['Temp'].str[-5:].astype(int).astype(str) + quarterly_df['Ticker'].str[-2:]
                        quarterly_df['Ticker'] = quarterly_df['Symbol']
                        # display(quarterly_df)
                        quarterly_df = quarterly_df.drop(quarterly_df.columns[9:],axis=1)
                        quarterly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Quarterly\\NIFTY-{file}.csv',index=False)
                        quarter_month_counter = group.iloc[0]['Exp_month']
                        quarter_year_counter = group.iloc[0]['Exp_year']
                        quarter +=1
    print("Quarterly Contracts Created")
    return quarterly_final_df

def nifty_halfyearly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\'+files)
        
    half_yearly = nifty_quarterly()
    half_yearly = half_yearly[(half_yearly['Exp_month'] == 6) | (half_yearly['Exp_month'] == 12)].reset_index(drop=True)
    half_year = 0
    halfyear_check = 6
    for name, group in tqdm(half_yearly.groupby(['Exp_year','Exp_month'])):
        # print(name)
        if half_year == 0:
            print(f"HalfYearly{half_year+1}")
            half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # file = np.where(half_year==0,'I',np.where(half_year==1,'II',np.where(half_year==2,'III',np.where(half_year==3,'IV',np.where(half_year==4,'V',np.where(half_year==5,'VI',np.where(half_year==6,'VII',np.where(half_year==7,'VIII',np.where(half_year==8,'IX',np.where(half_year==9,'X',''))))))))))
            # half_yearly_df['Symbol'] = 'NIFTY-' + str(file) + half_yearly_df['Temp'].str[-5:].astype(int).astype(str) + half_yearly_df['Ticker'].str[-2:]
            # half_yearly_df['Ticker'] = half_yearly_df['Symbol']
            # display(half_yearly_df)
            half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
            half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_year+1}.csv',index=False)
            halfyear_month_counter = group.iloc[0]['Exp_month']
            halfyear_year_counter = group.iloc[0]['Exp_year']
            half_year +=1
        else:
            if halfyear_month_counter == 12:
                halfyear_month_counter = 0
            if group.iloc[0]['Exp_month'] - halfyear_month_counter == halfyear_check:
                print(f"HalfYearly{half_year+1}")
                half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                # file = np.where(half_year==0,'I',np.where(half_year==1,'II',np.where(half_year==2,'III',np.where(half_year==3,'IV',np.where(half_year==4,'V',np.where(half_year==5,'VI',np.where(half_year==6,'VII',np.where(half_year==7,'VIII',np.where(half_year==8,'IX',np.where(half_year==9,'X',''))))))))))
                # half_yearly_df['Symbol'] = 'NIFTY-' + str(file) + half_yearly_df['Temp'].str[-5:].astype(int).astype(str) + half_yearly_df['Ticker'].str[-2:]
                # half_yearly_df['Ticker'] = half_yearly_df['Symbol']
                # display(half_yearly_df)
                half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
                half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_year+1}.csv',index=False)
                halfyear_month_counter = group.iloc[0]['Exp_month']
                halfyear_year_counter = group.iloc[0]['Exp_year']
                half_year +=1
            else:
                while(half_year<10):
                    if group.iloc[0]['Exp_year'] - halfyear_year_counter == 2:
                        half_year += 2
                    elif group.iloc[0]['Exp_year'] - halfyear_year_counter == 3:
                        half_year += 3
                    else:
                        half_year +=1
                    halfyear_check+=6
                    if group.iloc[0]['Exp_month'] - halfyear_month_counter == halfyear_check:
                        print(f"HalfYearly{half_year+1}")
                        half_yearly_df = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
                        # file = np.where(half_year==0,'I',np.where(half_year==1,'II',np.where(half_year==2,'III',np.where(half_year==3,'IV',np.where(half_year==4,'V',np.where(half_year==5,'VI',np.where(half_year==6,'VII',np.where(half_year==7,'VIII',np.where(half_year==8,'IX',np.where(half_year==9,'X',''))))))))))
                        # half_yearly_df['Symbol'] = 'NIFTY-' + str(file) + half_yearly_df['Temp'].str[-5:].astype(int).astype(str) + half_yearly_df['Ticker'].str[-2:]
                        # half_yearly_df['Ticker'] = half_yearly_df['Symbol']
                        # display(half_yearly_df)
                        half_yearly_df = half_yearly_df.drop(half_yearly_df.columns[9:],axis=1)
                        half_yearly_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_year+1}.csv',index=False)
                        halfyear_month_counter = group.iloc[0]['Exp_month']
                        halfyear_year_counter = group.iloc[0]['Exp_year']
                        halfyear_check=0
                        break
    print("Half Yearly contracts created!")

def nifty_yearly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Yearly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Yearly\\'+files)
        
    yearly_file = 1
    half_yearly_file1 = 1
    half_yearly_file2 = 2
    for i in range(5):
        # print(i,yearly_file,half_yearly_file1,half_yearly_file2)
        if os.path.exists(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_yearly_file1}.csv'):
            df1 = pd.read_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_yearly_file1}.csv')
        else:
            df1 = pd.DataFrame()
        if os.path.exists(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_yearly_file2}.csv'):
            df2 = pd.read_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{half_yearly_file2}.csv')
        else:
            df2 = pd.DataFrame()
        if df1.empty and df2.empty == True:
            print(f"No Contracts for HalfYearly {half_yearly_file1} {half_yearly_file2}")
        else:
            final_df = pd.DataFrame()
            if not df1.empty:
                df1['Temp'] = df1['Ticker'].str.replace('NIFTY','').str.replace('.NFO','').str[:-2]
                df1['Ticker_Expiry_Date'] = pd.to_datetime(df1['Temp'].str[:7],format='mixed',dayfirst=True)
                df1['Exp_month'] = df1['Ticker_Expiry_Date'].dt.month
                df1_new = df1[df1['Exp_month']==12].reset_index(drop=True)
            if not df2.empty:
                df2['Temp'] = df2['Ticker'].str.replace('NIFTY','').str.replace('.NFO','').str[:-2]
                df2['Ticker_Expiry_Date'] = pd.to_datetime(df2['Temp'].str[:7],format='mixed',dayfirst=True)
                df2['Exp_month'] = df2['Ticker_Expiry_Date'].dt.month
                df2_new = df2[df2['Exp_month']==12].reset_index(drop=True)
            final_df = pd.concat([df1_new,df2_new],ignore_index=True)
            
            if not df1.empty:
                file1 = np.where(half_yearly_file1==1,'I',np.where(half_yearly_file1==3,'III',np.where(half_yearly_file1==5,'V',np.where(half_yearly_file1==7,'VII',np.where(half_yearly_file1==9,'IX','')))))
                df1['Symbol'] = 'NIFTY-' + str(file1) + df1['Temp'].str[-5:].astype(int).astype(str) + df1['Ticker'].str[-2:]
                df1['Ticker'] = df1['Symbol']
                df1 = df1.drop(df1.columns[9:],axis=1)
                df1.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{file1}.csv',index=False)

            if not df2.empty:
                file2 = np.where(half_yearly_file2==2,'II',np.where(half_yearly_file2==4,'IV',np.where(half_yearly_file2==6,'VI',np.where(half_yearly_file2==8,'VIII',np.where(half_yearly_file2==10,'X','')))))
                df2['Symbol'] = 'NIFTY-' + str(file2) + df2['Temp'].str[-5:].astype(int).astype(str) + df2['Ticker'].str[-2:]
                df2['Ticker'] = df2['Symbol']
                df2 = df2.drop(df2.columns[9:],axis=1)
                df2.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Half_Yearly\\Nifty-{file2}.csv',index=False)
            
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V','')))))
            final_df['Symbol'] = 'NIFTY-' + str(file) + final_df['Temp'].str[-5:] + final_df['Ticker'].str[-2:]
            final_df['Ticker'] = final_df['Symbol']
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            final_df.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\Nifty\\Yearly\\Nifty-{file}.csv',index=False)
        yearly_file+=1
        half_yearly_file1+=2
        half_yearly_file2+=2
    print("Yearly Contracts Created")

def finnifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.1.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    #spark.sparkContext.setLogLevel("WARN")
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()
    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    ## FILTERING FINNIFTY DATA
    finnifty = q.filter(q.ticker.contains('FINNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    finnifty = finnifty.withColumn('ticker',regexp_replace('ticker','.NFO',''))
    finnifty = finnifty.drop(col('ticker_check'))
    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    finnifty = finnifty.toPandas()
    finnifty = finnifty.rename(columns={'ticker':'Ticker','date':'Date','time':'Time','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})

    # df = pd.read_csv(r"C:\\users\\admin\\desktop\\NSEFO_06092023.csv")
    # finnifty = df[(df['Ticker'].str.startswith('FINNIFTY') & (df['Ticker'].str.endswith('E.NFO')))].reset_index(drop=True)
    # finnifty['Ticker'] = finnifty['Ticker'].str.replace('.NFO','')
    
    finnifty = finnifty[(finnifty['Time']>='09:15:59') & (finnifty['Time']<='15:29:59')].reset_index(drop=True)
    finnifty['Date'] = pd.to_datetime(finnifty['Date'],format='mixed',dayfirst=True)
    finnifty['Ticker'] = finnifty['Ticker'].str.replace('.NFO','')
    finnifty['Temp'] = finnifty['Ticker'].str.replace('FINNIFTY','').str.replace('.NFO','').str[:-2]
    finnifty['Length_Of_Temp'] = np.where(finnifty['Temp'].str.len()==15,15,finnifty['Temp'].str.len())
    finnifty['Ticker_Expiry_Date'] = pd.to_datetime(finnifty['Temp'].str[:7],format='mixed',dayfirst=True)
    finnifty['Exp_year'] = finnifty['Ticker_Expiry_Date'].dt.year
    finnifty['Exp_month'] = finnifty['Ticker_Expiry_Date'].dt.month
    finnifty = finnifty[finnifty['Ticker_Expiry_Date']>=finnifty['Date']].reset_index(drop=True)                             ## filtering out wrong ticker dates
    finnifty['DayOfWeek'] = finnifty['Date'].dt.isocalendar().day
    finnifty['ExpiryDayOfWeek'] = finnifty['Ticker_Expiry_Date'].dt.isocalendar().day
    finnifty['Current_Week_number'] = finnifty['Date'].dt.isocalendar().week
    finnifty['Expiry_Week_number_check'] = finnifty['Ticker_Expiry_Date'].dt.isocalendar().week
    # banknifty.to_csv(r'C:\\users\\admin\\desktop\\banknifty1.csv',index=False)
    finnifty['Expiry_Week_number'] = np.where(finnifty['ExpiryDayOfWeek']<=2,finnifty['Ticker_Expiry_Date'].dt.isocalendar().week-1,finnifty['Ticker_Expiry_Date'].dt.isocalendar().week)
    finnifty['Expiry_Week_number'] = np.where((finnifty['DayOfWeek']<=2) & (finnifty['ExpiryDayOfWeek']<=2),finnifty['Expiry_Week_number']+1,finnifty['Expiry_Week_number'])
    finnifty['Week_Diff'] = finnifty['Expiry_Week_number'] - finnifty['Current_Week_number']
    finnifty["Week_Diff"] = finnifty['Week_Diff'].replace(np.nan,10000)
    return finnifty

def finnifty_weekly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Weekly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Weekly\\'+files)
        
    finnifty = finnifty_data()
    weekly1 = finnifty[finnifty['ExpiryDayOfWeek']<=2].reset_index(drop=True)
    agb = weekly1.groupby(["Week_Diff"])
    unique_val_list_a = list(weekly1["Week_Diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    for i in sorted(unique_val_list_a):
        if i <= 14:
            print(f'Week{i+1}')
            temp_df = agb.get_group(i)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III',np.where(i==3,'IV',np.where(i==4,'V',np.where(i==5,'VI',np.where(i==6,'VII',np.where(i==7,'VIII',np.where(i==8,'IX',np.where(i==9,'X',np.where(i==10,'XI',np.where(i==11,'XII',np.where(i==12,'XIII',np.where(i==13,'XIV',''))))))))))))))
            temp_df['Symbol'] = 'FINNIFTY' + 'WEEKLY-' + str(file) + temp_df['Temp'].str[-5:].astype(int).astype(str) + temp_df['Ticker'].str[-2:]
            temp_df['Ticker'] = temp_df['Symbol']
            temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
            temp_df = temp_df.drop_duplicates()
            temp_df.to_csv(fr'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Weekly\\FINNIFTY-{file}.csv',index=False)
    print("Weekly Contracts Created")

def finnifty_monthly():
    ## REMOVING ALREADY STORED FILES
    weekly_files = os.listdir(r'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Monthly')
    for files in weekly_files:
        os.remove(r'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Monthly\\'+files)
        
    finnifty = finnifty_data()
    i = 0
    monthly_df = pd.DataFrame()
    for name,group in tqdm(finnifty.groupby(['Exp_year','Exp_month'])):
        monthly_df = pd.concat([monthly_df,group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]],ignore_index=True)
        if i <3 :
            print(f"Monthly{i+1}")
            monthly = group[group['Ticker_Expiry_Date'] == group['Ticker_Expiry_Date'].max()]
            # display(monthly)
            file = np.where(i==0,'I',np.where(i==1,'II',np.where(i==2,'III','')))
            monthly['Symbol'] = 'FINNIFTY' + 'MONTHLY-' + str(file) + monthly['Temp'].str[-5:].astype(int).astype(str) + monthly['Ticker'].str[-2:]
            monthly['Ticker'] = monthly['Symbol']
            monthly = monthly.drop(monthly.columns[9:],axis=1)
            monthly.to_csv(rf'C:\\users\\admin\\desktop\\Pyspark\\FinNifty\\Monthly\\FINNIFTY-{file}.csv',index=False)
            i+=1
    print("Monthly Contracts Created")
    return monthly_df

print("BankNifty Contracts being created")
banknifty_weekly()
banknifty_halfyearly()
banknifty_yearly()
print("BankNifty Contracts created\n")
print("Nifty Contracts being created")
nifty_weekly()
nifty_halfyearly()
nifty_yearly()
print("Nifty Contracts created\n")
print("FinNifty Contracts being created")
finnifty_weekly()
finnifty_monthly()
print("FinNifty Contracts created\n")
# nifty_halfyearly()
# nifty_yearly()

et = time.time()

link1 = "BankNifty options creation finished!"
base_url =f"https://api.telegram.org/bot6237928541:AAHl267HrSFBRFE-iIajz_x8eNkPydiQEEs/sendMessage?chat_id=-939411532&text={link1}"
requests.get(base_url)

print("\nElapsed time",et-st)

Enter a date in YYYY-MM-DD format  2023-09-06


r06092023
BankNifty Contracts being created
Week1
Week2
Week3
Week5


Week4
Week8
Week13
Weekly Contracts Created!


  0%|          | 0/4 [00:00<?, ?it/s]

Monthly1
Monthly2
Monthly3
Monthly Contracts Created


  0%|          | 0/2 [00:00<?, ?it/s]

...

Quarter1
Quarter2
Quarter3
Quarter4
Quarterly Contracts Created


  0%|          | 0/4 [00:00<?, ?it/s]

HalfYearly1
HalfYearly2
HalfYearly3
HalfYearly5
Half Yearly contracts created!
No Contracts for HalfYearly 7 8
No Contracts for HalfYearly 9 10
Yearly Contracts Created
Nifty Contracts created

FinNifty Contracts being created
Week1
Week2
Week3
Weekly Contracts Created


  0%|          | 0/1 [00:00<?, ?it/s]

Monthly1
Monthly Contracts Created
FinNifty Contracts created


Elapsed time 18.966927528381348
